In [80]:
from functools import lru_cache


sample = """[1,1,3,1,1]
[1,1,5,1,1]

[[1],[2,3,4]]
[[1],4]

[9]
[[8,7,6]]

[[4,4],4,4]
[[4,4],4,4,4]

[7,7,7,7]
[7,7,7]

[]
[3]

[[[]]]
[[]]

[1,[2,[3,[4,[5,6,7]]]],8,9]
[1,[2,[3,[4,[5,6,0]]]],8,9]"""

real = open('d13.input').read()


def get_pairs(data):
    pairs = {}
    for i, pair in enumerate(data.split("\n\n"), start=1):
        left, right = pair.split()
        pairs[i] = (eval(left), eval(right))
    return pairs
        

def get_list(data):
    entries = []
    for i, entry in enumerate(data.split("\n"), start=1):
        if not entry:
            continue
        entries.append(entry)
    return entries


get_pairs(sample)

print(len(get_list(sample)))


16


In [86]:
from itertools import zip_longest
DEBUG = False

DUNNO = object()

def compare_numbers(left, right, pad):
    for l, r in zip_longest(left, right):
        if l is None:
            DEBUG and print(pad + f"{l=} {r=}.  Left None. Returning True")
            return True
        if r is None:
            DEBUG and print(pad + f"{l=} {r=}.  Right None. Returning False")
            return False
        if l == r:
            DEBUG and print(pad + f"{l=} {r=}.  Equal.  Continuing")
            continue
        if l < r:
            DEBUG and print(pad + f"{l=} {r=}.  Left less.  True")
            return True
        if l > r:
            DEBUG and print(pad + f"{l=} {r=}.  Left more.  False")
            return False
    return DUNNO

no_lists = lambda l: not bool([e for e in l if isinstance(e, list)])

def is_sorted(left, right, depth=0):
    pad = depth * " "
    depth+=1
    left_is_int = isinstance(left, int)
    right_is_int = isinstance(right, int)
    DEBUG and print(pad + f'Comparing {left=} {right=}')
    if left_is_int and right_is_int:
        DEBUG and print(pad + f'Both numbers')
        return compare_numbers([left],[right], pad)
    if left_is_int:
        DEBUG and print(pad + f'converting left to [{left}]')
        left = [left]
    if right_is_int:
        DEBUG and print(pad + f'converting right to [{right}]')
        right = [right]

    if no_lists(left) and no_lists(right):
        DEBUG and print(pad + f'all numbers.  Sending to compare_numbers')
        return compare_numbers(left, right, pad)    
    
    for l, r in zip_longest(left, right):
        DEBUG and print(pad + f'Recursing {l=} {r=}')
        if l is None:
            DEBUG and print(pad + f'Left ran out of items. True')
            return True
        if r is None:
            DEBUG and print(pad + f'Right ran out of items. False')
            return False
        result = is_sorted(l,r, depth)
        if result in (True, False):
            return result
    else:
        DEBUG and print(pad + f'All recursed for {left=} {right=}.  {result=}')
        return result


@lru_cache(maxsize=None)
def cached_is_sorted(string1, string2):
    return is_sorted(eval(string1), eval(string2))

def get_index_sums(data):
    global DEBUG
    pairs = get_pairs(data)
    in_order = 0
    results = {}
    for i,(left, right) in pairs.items():
        DEBUG and print("")
        result = is_sorted(left, right)
        if result:
            in_order += i
            results[i] = True
        DEBUG = False
    print(sum(results))

    for i,(left, right) in pairs.items():
        if not DEBUG:
            break
        print (f'Left: {left}')
        print (f'Right: {right}')
        print(f'Result={results.get(i, False)}')
        print("")






In [82]:
DEBUG = False
get_index_sums(sample)
get_index_sums(real)

13
5013


In [87]:
def sort_entries(data):
    TWO = "[[2]]"
    SIX = "[[6]]"
    entries = [TWO,SIX]
    entries.extend(get_list(data))
    elen = len(entries)
    previous = entries.copy()
    i = 0
    start = 0
    sweep = list(zip(range(start,elen-1), range(start+1,elen)))
    while True:
        for ei, fi in sweep:
            i += 1
            if not cached_is_sorted(entries[ei], entries[fi]):
                DEBUG and print(f"Swapping {ei} and {fi}")
                entries[ei],entries[fi] = entries[fi],entries[ei]
        if entries == previous:
            break
        previous = entries.copy()
    print(f"Sorted in {i} steps")
    print ((entries.index(TWO) + 1) * (entries.index(SIX) + 1))
    


DEBUG = False
sort_entries(sample)
sort_entries(real)

        


Sorted in 255 steps
140
Sorted in 87892 steps
25038
